In [66]:
import numpy as np
#Molar Ratios Calculation: Calc Fe/Mg and Si/Mg from [Fe/H], [Mg/H], and [Si/H]. 
FeH = 0.16       #From top of SPECIES
MgH = -0.23    
SiH = 0.01 
sigFeH = 0.045 #From top of SPECIES
sigMgH = 0.141
sigSiH = 0.082

#Ref solar values from Lodders 2009
FeH_sun = 7.45; SiH_sun = 7.52; MgH_sun = 7.54
#sigFeH_sun = 0.03; sigSiH_sun = 0.02; sigMgH_sun = 0.02
sigFeH_sun = 0.08; sigSiH_sun = 0.06; sigMgH_sun = 0.06
#sigFeH_sun = 0.0; sigSiH_sun = 0.0; sigMgH_sun = 0.0
        
FeMg =  10**(FeH+FeH_sun-MgH-MgH_sun)
SiMg = 10**(SiH+SiH_sun-MgH-MgH_sun)
    
sigFeMg = FeMg*np.log(10.0)*np.sqrt(sigFeH**2 + sigMgH**2 + sigFeH_sun**2 + sigMgH_sun**2)
sigSiMg = SiMg*np.log(10.0)*np.sqrt(sigSiH**2 + sigMgH**2 + sigSiH_sun**2 + sigMgH_sun**2)

    
print('Fe/Mg is : '+str(FeMg)+' with uncertainty '+str(sigFeMg))
print('Si/Mg is : '+str(SiMg)+' with uncertainty '+str(sigSiMg))


Fe/Mg is : 1.9952623149688828 with uncertainty 0.8206384645312165
Si/Mg is : 1.6595869074375598 with uncertainty 0.7025967860601234


In [67]:
#Schulze Star CMF Calculation
mfe = 55.845; msio2 = 60.08; mmgo = 40.3044
cmfstar = FeMg*mfe/(FeMg*mfe + SiMg*msio2 + mmgo)
    
sigcmfstar = (FeMg*msio2*sigSiMg)**2 + (sigFeMg*(mmgo + msio2*SiMg))**2 
sigcmfstar = mfe*np.sqrt(sigcmfstar)/((FeMg*mfe + mmgo + msio2*SiMg)**2)
    
print('Star CMF is: ' + str(100*cmfstar) + ' with uncertainty ' + str(100*sigcmfstar))

Star CMF is: 44.315302470843136 with uncertainty 12.584154276577053


In [68]:
import numpy as np
import scipy.stats as sp
from scipy.integrate import quad
radius = [1.626, 0.051] #In Terms of Earth Radii
mass = [5.59, 0.97] #In Terms of Earth Masses

 
#Define CMF function and density functions

#General CMF function in terms of planetary observables -- Mass and Radius. 
def cmf_rho(M, R):
    cmfrho = (4./3.)*np.pi*(R**3)*rhoc(M) - M*rhoc(M)/rhom(M)
    cmfrho = cmfrho/(M*(1-rhoc(M)/rhom(M)))
    return 100*cmfrho


#general density function 
def rho(M,a0,a1,a2):
    rho = a0+(a1*(M**a2))
    return rho


#Build functions for the bulk core density and bulk mantle density using the 
#general density function. The parameters being fed into the rho function were
#found from fits of rhoc vs Mp and rhom vs Mp using exoplex over the mass
#range of 0.01 Earth masses to 11 Earth masses.
#   rhoc: bulk core density
#   rhom: bulk mantle density
def rhoc(Mp):
    rhoc = rho(Mp, 0.36125593204992873, 0.153364551060784, 0.5351858792798839)
    return rhoc


def rhom(Mp):

    rhom = rho(Mp, 0.12372222237561939, 0.06565578408094341, 0.4096898631741826)
    return rhom



#calc_cmfrho mostly calculates the 1-sigma uncertainty in CMF rho. It does so
#by first generating the 68% confidence ellipse of the joint mass-radius distribution.
#It then feeds this ellipse into cmf_rho to generate the 68% confidence interval
#on CMF rho. Finally it averages all of the values along this ellipse that yield
#a CMF that is greater than the mean CMF value to give a scalar value for the 
#upper uncertainty in CMF rho. Similarly, it averages all of the CMF values along
#this ellipse that yield a value that is less than the mean CMF to calculate
#a scalar approximation for the lower uncertainty on CMF rho.
    
def calc_cmfrho(mass, radius):
    
    #unpack mass and radius
    Mass = mass[0]; sigM = mass[1]
    Radius = radius[0]; sigR = radius[1]
    
    #ellipse angle array
    angle = np.linspace(0.0,2*np.pi,1000)
    
    #68% confidence ellipse
    csquared68 = sp.chi2.ppf(0.68,2)
    
    
    #Calc semi-major and semi-minor axes.
    rrrr =(csquared68**0.5)*sigR*np.cos(angle)+Radius
    rmrm = (csquared68**0.5)*sigM*np.sin(angle)+Mass
    
    #calculate the difference of between all points on the CMF ellipse 
    #and the mean CMF value.
    cmfdif = cmf_rho(rmrm, rrrr) - cmf_rho(Mass, Radius)*np.ones(len(rrrr))

    #The next block of code takes the averages above and below the mean
    #CMF value to approximate the upper and lower uncertanties in CMF rho.
    sigmp = 0
    sigmm = 0
    sigrp = 0
    sigrm = 0

    countp = 0
    countm = 0

    for j in range(0, len(rrrr)):
        if cmfdif[j]>0:
            sigmp = sigmp + rmrm[j]
            sigrp = sigrp + rrrr[j]
            countp = countp+1
        if cmfdif[j]<0:
            sigmm = sigmm + rmrm[j]
            sigrm = sigrm + rrrr[j]
            countm = countm+1
        

        
    sigmm = sigmm/countm
    sigrm = sigrm/countm

    sigrp = sigrp/countp
    sigmp = sigmp/countp
    
    #print ('M upper: ', sigmp)
    #print ('R upper: ', sigrp)
    
    #'M lower: ', sigmm
    #'R lower: ', sigrm
    
    
    cmfrho = cmf_rho(Mass, Radius)
    
    sigcmfp_val = abs(cmfrho-cmf_rho(sigmp,sigrp))
    sigcmfm_val = abs(cmfrho-cmf_rho(sigmm,sigrm))
    
    return cmfrho, sigcmfp_val, sigcmfm_val
    
calc_cmfrho(mass, radius)


(26.418855779254436, 17.918246263488378, 22.5182448522866)

In [69]:
#Adibekyan Star CMF Calculation
import math

PYROXENE_MOLAR_MASS = 100.3887
OLIVINE_MOLAR_MASS = 140.6931
IRON_MOLAR_MASS = 55.845
SILICON_DIOXIDE_MOLAR_MASS = 60.0843

Fe_num = 10**(FeH+math.log(FeH_sun, 10))
Fe_num_sig = math.sqrt((FeH_sun*(10**(FeH))*math.log(10)*(sigFeH))**2)
Mg_num = 10**(MgH+math.log(MgH_sun, 10))
Mg_num_sig = math.sqrt((MgH_sun*(10**(MgH))*math.log(10)*(sigMgH))**2)
Si_num = 10**(SiH+math.log(SiH_sun, 10))
Si_num_sig = math.sqrt((SiH_sun*(10**(SiH))*math.log(10)*(sigSiH))**2)
print(Mg_num/Si_num)

if Mg_num/Si_num > 1 and Mg_num/Si_num < 2:
    Pyroxene_num = (2*Si_num)-Mg_num
    Pyroxene_num_sig = math.sqrt((4*(Si_num_sig**2))+(Mg_num_sig**2))
    Olivine_num = Mg_num - Si_num
    Olivine_num_sig = math.sqrt((Mg_num_sig**2)+(Si_num_sig**2))
    cmf_star = (IRON_MOLAR_MASS*Fe_num)/((IRON_MOLAR_MASS*Fe_num)+(PYROXENE_MOLAR_MASS*Pyroxene_num)+(OLIVINE_MOLAR_MASS*Olivine_num))
    print('Star CMF is: ' + str(cmf_star))
elif Mg_num/Si_num < 1:
    Pyroxene_num = Mg_num
    Pyroxene_num_sig = math.sqrt(Mg_num_sig**2)
    Silicon_dioxide_num = Si_num - Mg_num
    Silicon_dioxide_num_sig = math.sqrt((Mg_num_sig**2)+(Si_num_sig**2))
    cmf_star = (IRON_MOLAR_MASS*Fe_num)/((IRON_MOLAR_MASS*Fe_num)+(PYROXENE_MOLAR_MASS*Pyroxene_num)+(SILICON_DIOXIDE_MOLAR_MASS*Silicon_dioxide_num))
    print('Star CMF is: ' + str(cmf_star))
else:
    print('Ratio greater than 2')
    
x = IRON_MOLAR_MASS*Fe_num
y = PYROXENE_MOLAR_MASS*Pyroxene_num
#z = OLIVINE_MOLAR_MASS*Olivine_num
d = SILICON_DIOXIDE_MOLAR_MASS*Silicon_dioxide_num

iron = (d+y)/((x+y+d)**2)
iron2 = iron * Fe_num_sig

pyroxene = x/((x+y+d)**2)
pyroxene2 = pyroxene * Pyroxene_num_sig

#olivine = x/((x+y+z)**2)
#olivine2 = olivine * Olivine_num_sig

sio2 = x/((x+y+d)**2)
sio2_2 = sio2 * Silicon_dioxide_num_sig

if Mg_num/Si_num > 1 and Mg_num/Si_num < 2:
    uncertainty_final = math.sqrt((iron2**2)+(pyroxene2**2)+(olivine2**2))
    # cmf_star_sig = math.sqrt(((IRON_MOLAR_MASS**2)*(Fe_num_sig**2))/(((IRON_MOLAR_MASS**2)*(Fe_num_sig**2))+((PYROXENE_MOLAR_MASS**2)*(Pyroxene_num_sig**2))+((OLIVINE_MOLAR_MASS**2)*(Olivine_num_sig**2))))
    print('Uncertainty is: ' + str(uncertainty_final*100))
elif Mg_num/Si_num < 1:
    uncertainty_final = math.sqrt((iron2**2)+(pyroxene2**2)+(sio2_2**2))
    # cmf_star_sig = math.sqrt(((IRON_MOLAR_MASS**2)*(Fe_num_sig**2))/(((IRON_MOLAR_MASS**2)*(Fe_num_sig**2))+((PYROXENE_MOLAR_MASS**2)*(Pyroxene_num_sig**2))+((OLIVINE_MOLAR_MASS**2)*(Olivine_num_sig**2))))
    print('Uncertainty is: ' + str(uncertainty_final*100))
    

0.5769703627024155
Star CMF is: 0.4839310596256296
Uncertainty is: 0.10793928116300863
